In [1]:
import torch
from torchvision import models
from simplify import simplify
import time
from tqdm import tqdm
import torch
from torch import nn
import torch.nn.utils.prune as prune
import torch.nn.functional as F

In [2]:
model = models.resnet18(pretrained=True)
# print('model all param name :', [name for name, _ in model.named_parameters()])
dummy_input = torch.zeros(64, 3, 224,224)  # Tensor shape is that of a standard input for the given model
def run_model(model, dummy_input):
    start = time.time()
    for _ in tqdm(range(10)):
        model(dummy_input)
    end = time.time()
    print("Time taken: ", end - start)

print("Original model")
run_model(model, dummy_input)

/home/zeyuan.yin/.conda/envs/torch/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/home/zeyuan.yin/.conda/envs/torch/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Original model


100%|██████████| 10/10 [00:10<00:00,  1.01s/it]

Time taken:  10.154376029968262


In [3]:
model.half()
print("half model")
run_model(model, dummy_input.half())

half model


  0%|          | 0/10 [00:01<?, ?it/s]


RuntimeError: Input type (torch.FloatTensor) and weight type (torch.HalfTensor) should be the same or input should be a MKLDNN tensor and weight is a dense tensor

In [ ]:
model = models.resnet18(pretrained=True) # reset model
import torch.quantization

quantized_model = torch.quantization.quantize_dynamic(
    model, {torch.nn.Conv2d,torch.nn.Linear}, dtype=torch.qint8
)

print("Quantized model")
run_model(quantized_model, dummy_input)

Quantized model


100%|██████████| 10/10 [00:10<00:00,  1.00s/it]

Time taken:  10.026480913162231
